In [1]:
import pandas as pd

loan_lender = pd.read_csv('loans_lenders.csv')
loan = pd.read_csv('loans.csv')
lenders = pd.read_csv('lenders.csv')
#mostra tutte le colonne quando faccio .head()
pd.set_option('display.max_columns', None)


In [2]:
#1 Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.
#Trasformo la stringa contenente i nomi dei lender in una lista contenente i medesimi nomi
loan_lender['lenders'] = loan_lender['lenders'].str.split(', ')
#creo un dataframe dove inserirò le nuove rows normalizzate
loan_lenderNorm = pd.DataFrame(columns = ['loan_id', 'lenders']) 
#utilizzo il metodo explode sulla colonna 'lenders' per trasformare ogni elemento della lista lenders in una riga
loan_lenderNorm = loan_lender.explode('lenders')
loan_lenderNorm.reset_index(drop=True, inplace=True)
loan_lenderNorm

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


In [3]:
#2.For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time.
#i valori mancanti in loan planned_expiration_time e disburse_time sono NaT ma vengono valutati come valori .isnull()
# quando viene effettuata la sottrazione tra un datetime e un valore NaT la differenza viene automaticamente uguale a NaN
loan['disburse_time'] = pd.to_datetime(loan['disburse_time'], format = '%Y-%m-%d %H:%M:%S.%f %z')
loan['planned_expiration_time'] = pd.to_datetime(loan['planned_expiration_time'], format = '%Y-%m-%d %H:%M:%S.%f %z')
loan['duration'] = (loan['planned_expiration_time'] - loan['disburse_time']).dt.days
loan.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,53.0
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,96.0
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37.0
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34.0
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57.0


In [4]:
#3.Find the lenders that have funded at least twice.
#creo un dizionario key-value con chiave il nome del lenders e valore il n° prestiti
dict= {}
for i in loan_lenderNorm['lenders']:
    if i not in dict.keys():
        dict[i]=1
    else:
        dict[i]= dict[i]+1
#utilizzo una dict comprehension per estrarre le coppie chiave-valore con valore > 1
dict3 = { key:value for key, value in dict.items() if value > 1 }
#inserisco i nomi dei lenders in una lista
lenders2founded = list(dict3.keys())
lenders2founded

['muc888',
 'sam4326',
 'camaran3922',
 'rebecca3499',
 'karlheinz4543',
 'jerrydb',
 'paula8951',
 'gmct',
 'amra9383',
 'r3922',
 'brian9451',
 'shree8053',
 'alan5513',
 'oisin3389',
 'helle8622',
 'bo3186',
 'ric8947',
 'daniel98469874',
 'nick9464',
 'deborah12671549',
 'matthew9831',
 'john6330',
 'john9479',
 'mattiaslaven',
 'jonathan2867',
 'jason3883',
 'highgrovechurch',
 'maria3124',
 'dino5102',
 'jonathan7946',
 'ann8187',
 'bryan2669',
 'john88459657',
 'eddyphil',
 'don9212',
 'carolineandcolin9686',
 'bent8782',
 'raph8817',
 'danielle2350',
 'nora3555',
 'williammanashi',
 'barbara5610',
 'danhostetler',
 'daniel1104',
 'amirali5409',
 'oceanwest',
 'greg69396003',
 'trolltech4460',
 'thedragonflykeeper',
 'patrick8466',
 'priscilla3168',
 'terrystl',
 'richardandsusan8352',
 'sherri4341',
 'gooddogg1',
 'danny6470',
 'jacqueline4838',
 'diederik8163',
 'ryan54597608',
 'james5068',
 'rudi5955',
 'daniel9859',
 'chandrakant6134',
 'karyne5301',
 'robert2280',
 'darrel

In [5]:
#4.For each country, compute how many loans have involved that country as borrowers.
loan.groupby('country_code').count()['loan_id']

country_code
AF     2337
AL     3075
AM    13952
AZ    10172
BA      608
      ...  
XK     2178
YE     4206
ZA      633
ZM     1277
ZW     5513
Name: loan_id, Length: 95, dtype: int64

In [6]:
#5. For each country, compute the overall amount of money borrowed.
country_loan_amount= loan.groupby('country_code').sum()['loan_amount']
country_loan_amount

country_code
AF     1967950.0
AL     4307350.0
AM    22950475.0
AZ    14784625.0
BA      477250.0
         ...    
XK     3083025.0
YE     3444000.0
ZA     1006525.0
ZM     1978975.0
ZW     5851875.0
Name: loan_amount, Length: 95, dtype: float64

In [7]:
#6.Like the previous point, but expressed as a percentage of the overall amount lent.
totale_prestato = sum(loan['loan_amount'])
perc_country_loan = (country_loan_amount*100)/totale_prestato
perc_country_loan

country_code
AF    0.166573
AL    0.364586
AM    1.942589
AZ    1.251410
BA    0.040396
        ...   
XK    0.260955
YE    0.291509
ZA    0.085195
ZM    0.167506
ZW    0.495318
Name: loan_amount, Length: 95, dtype: float64

In [8]:
#7.Like the three previous points, but split for each year (with respect to disburse time).
loan['disburse_time_year'] = loan['disburse_time'].dt.year
#For each country, compute how many loans have involved that country as borrowers but split for each year
loan.groupby(['country_code', 'disburse_time_year' ]).count()['loan_id']


country_code  disburse_time_year
AF            2007.0                 408
              2008.0                 370
              2009.0                 678
              2010.0                 632
              2011.0                 247
                                    ... 
ZW            2013.0                 426
              2014.0                2078
              2015.0                 600
              2016.0                 808
              2017.0                1079
Name: loan_id, Length: 745, dtype: int64

In [9]:
#For each country, compute the overall amount of money borrowed but split for each year
country_loan_amount_for_year= loan.groupby(['country_code', 'disburse_time_year']).sum()['loan_amount']
country_loan_amount_for_year

country_code  disburse_time_year
AF            2007.0                 194975.0
              2008.0                 365375.0
              2009.0                 585125.0
              2010.0                 563350.0
              2011.0                 245125.0
                                      ...    
ZW            2013.0                 678525.0
              2014.0                1311575.0
              2015.0                 723625.0
              2016.0                 788600.0
              2017.0                1237600.0
Name: loan_amount, Length: 745, dtype: float64

In [10]:
#Like the previous point, but expressed as a percentage of the overall amount lent and split for each year
perc_country_loan_for_year = (country_loan_amount_for_year*100)/totale_prestato
perc_country_loan_for_year

country_code  disburse_time_year
AF            2007.0                0.016503
              2008.0                0.030926
              2009.0                0.049527
              2010.0                0.047683
              2011.0                0.020748
                                      ...   
ZW            2013.0                0.057432
              2014.0                0.111015
              2015.0                0.061250
              2016.0                0.066749
              2017.0                0.104754
Name: loan_amount, Length: 745, dtype: float64

In [11]:
#8 For each lender, compute the overall amount of money lent.
#Calcolo la quantità prestata da ogni lender per quel prestito. Supponiamo che ogni lender contribuisca equamente
loan['loan_amount_4_lender'] = loan['loan_amount'] / loan['num_lenders_total']
#re-imposto gli indici e li ordino per velocizzare il join
loan_lender.set_index('loan_id', inplace=True)
loan.set_index('loan_id', inplace=True)
loan_lender.sort_index(inplace = True)
loan.sort_index(inplace = True)
#faccio il join sulla colonna loan_id. 
#Questa join mi serve per unire la lista dei lender per ogni loan alla quantità di denaro prestata.
#Ho ripreso la tabella loan_lender originale ed effettuato l'explode dopo per velocizzare l'operazione di join.
dfJoin = loan_lender.join(loan)
#estraggo solo le colonne che mi interessano, loan_id viene estratta automaticamente perché indice
new = dfJoin[['loan_amount_4_lender','lenders']]
new.reset_index(inplace = True)
new = new.explode('lenders')
df_lend_amount = new.groupby('lenders').sum()['loan_amount_4_lender']
df_lend_amount

In [24]:
#9 For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed.
#Since the country of the lender is often unknown, you can assume that the true distribution among the countries 
#is the same as the one computed from the rows where the country is known.

In [13]:
import pandas as pd
import numpy as np
#calcolo le frequenze relative di ogni country_code
s = lenders['country_code'].value_counts(normalize=True)
#check dove è null il valore di country code
missing = lenders['country_code'].isnull()
#Rimpiazzo dei valori mancanti scegliendo tra gli s.index (elenco country_code), tante volte quanti sono i val mancanti e 
#scegliendoli con una probabilità = alla freq relativa calcolata precedentemente (s.values)
lenders.loc[missing,'country_code'] = np.random.choice(s.index, size=len(lenders[missing]),p =s.values)

In [14]:
#Riprendo il risultato del punto precedente dove per ogni lenders ho la quantità di soldi prestata e lo trasformo in un 
#dataframe
df_lend_amount = df_lend_amount.to_frame()
#importo l'indice la colonna su cui farò il join e riordino gli indici
lenders.set_index('permanent_name', inplace = True)
df_lend_amount.sort_index(inplace = True)
lenders.sort_index(inplace = True)
country_lenders_amount = df_lend_amount.join(lenders)
country_lenders_amount = country_lenders_amount.groupby('country_code').sum()['loan_amount_4_lender']


In [15]:
country_lenders_amount = country_lenders_amount.to_frame()
country_lenders_amount.columns = ['lent_amount_4_country']
country_loan_amount = country_loan_amount.to_frame()
dfDiff = country_lenders_amount.join(country_loan_amount)
dfDiff = dfDiff.fillna(0)
#Aggiungo la colonna 'differenza' che è la differenza tra i soldi prestati e presi in prestito
dfDiff['differenza'] = dfDiff['lent_amount_4_country']- dfDiff['loan_amount']
dfDiff.head()

,lent_amount_4_country,loan_amount,differenza
country_code,,,
AD,5.200657e+03,0.0,5.200657e+03
AE,1.757587e+06,0.0,1.757587e+06
AF,1.249545e+05,1967950.0,-1.842996e+06
AG,4.426660e+03,0.0,4.426660e+03
AI,3.524281e+02,0.0,3.524281e+02


In [16]:
#10.Which country has the highest ratio between the difference computed at the previous point and the population?

country = pd.read_csv('country_stats.csv')
country1 = country.join(dfDiff, on='country_code')
country1['ratio'] = country1['population']/country1['differenza']
country1.loc[country1['ratio'].idxmax()]

country_name                              Niger
country_code                                 NE
country_code3                               NER
continent                                Africa
region                           Western Africa
population                             21477348
population_below_poverty_line              45.4
hdi                                    0.352641
life_expectancy                          61.936
expected_years_of_schooling             5.42265
mean_years_of_schooling                   1.658
gni                                     889.456
kiva_country_name                         Niger
lent_amount_4_country                   182.531
loan_amount                                   0
differenza                              182.531
ratio                                    117664
Name: 38, dtype: object

In [17]:
#11. Which country has the highest ratio between the difference computed at point 9 and the population 
#that is not below the poverty line?
country1['pop_not_below_poverty_line'] = country1['population']-((country1['population']*country1['population_below_poverty_line']) / 100)
country1['ratio_poverty_line'] = country1['pop_not_below_poverty_line']/country1['differenza']
country1.loc[country1['ratio_poverty_line'].idxmax()]

country_name                              Niger
country_code                                 NE
country_code3                               NER
continent                                Africa
region                           Western Africa
population                             21477348
population_below_poverty_line              45.4
hdi                                    0.352641
life_expectancy                          61.936
expected_years_of_schooling             5.42265
mean_years_of_schooling                   1.658
gni                                     889.456
kiva_country_name                         Niger
lent_amount_4_country                   182.531
loan_amount                                   0
differenza                              182.531
ratio                                    117664
pop_not_below_poverty_line          1.17266e+07
ratio_poverty_line                      64244.5
Name: 38, dtype: object

In [18]:
#12.For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different 
#years must have its amount distributed proportionally to the number of days in each year. 

In [19]:
#Divido il problema in più sottoproblemi, inizialmente chi ha l'anno uguale e chi no. 
#In quelli che ce l'hanno diverso abbiamo anche valori nulli che gestiremo dopo.
#Ho diviso il problema per renderlo più facilmente modificabile e leggibile

#creo la colonna booleana 'annoPrestito' che è True se disburse e expiration hanno lo stesso anno altrimento false 
loan['annoPrestito'] = (loan['disburse_time'].dt.year == loan['planned_expiration_time'].dt.year)
#inserisco in righe i loan con lo stesso anno
righe = loan.loc[loan['annoPrestito'] == True]
#inserisco in righe1 i loan con anni differenti
righe1 = loan.loc[loan['annoPrestito'] == False]

In [20]:
#Gestisco quelli che hanno lo stesso anno e NON considero i loan che hanno disburse time maggiore di expiration time
righe = righe.loc[righe['duration']>=0]
righe.reset_index(inplace = True)
#quantità di soldi prestati per anno con disburse_time uguale a quello di planned_Exp_time
risultato = righe.groupby(righe.disburse_time.dt.year).sum()['loan_amount']

In [21]:
#Gestisco i loan che hanno anni differenti

#Considero solo quelli che hanno disburse e expiration non nulli e con disburse time minore di expiration time
righe3 = righe1.loc[(pd.notnull(righe1['duration']))]
righe3 = righe3.loc[righe3['duration']>=0]

In [22]:
#creo un dizionario dove metto per ogni anno il valore speso
#lo creo in modo che sia scalabile nel caso cambiassero gli anni
diz={}
for x in range(min(righe3['disburse_time'].dt.year),max(righe3['planned_expiration_time'].dt.year)+1):
    if x not in diz.keys():
        diz[x]=0

#Suddivido il problema ancora in 2: quelli che hanno l'anno di disburse time precedente a quello di expiration time e quelli
#che hanno più anni di differenza

#Estraggo i prestiti che hanno l'anno di disburse time precedente a quello di expiration time
righe4 = (righe3.loc[righe3['planned_expiration_time'].dt.year-righe3['disburse_time'].dt.year==1])
prezzo = 0
prezzo1 = 0
for x in righe4.itertuples():
    giorniDisburse = 365-(x.disburse_time.dayofyear)
    giorniPlanned = x.disburse_time.dayofyear
    prezzo =  x.loan_amount * (giorniDisburse/x.duration)
    prezzo1 = x.loan_amount * (giorniPlanned/x.duration)
    diz[x.disburse_time.year]+=prezzo
    diz[x.planned_expiration_time.year] += prezzo1

    
#Estraggo i prestiti che hanno più di un anno di differenza tra disburse e expiration
righe5 = (righe3.loc[righe3['planned_expiration_time'].dt.year-righe3['disburse_time'].dt.year>1])
for x in righe5.itertuples():
    #Gestisco l'anno di inizio e di fine come prima
    giorniDisburse = 365-(x.disburse_time.dayofyear)
    giorniPlanned = x.disburse_time.dayofyear
    prezzo =  x.loan_amount * (giorniDisburse/x.duration)
    prezzo1 = x.loan_amount * (giorniPlanned/x.duration)
    diz[x.disburse_time.year]+=prezzo
    diz[x.planned_expiration_time.year] += prezzo1
    #Gestione anni intermedi 
    for anno in range (x.disburse_time.year+1, x.planned_expiration_time.year ):
        prezzo2 = x.loan_amount * (365/x.duration)
        diz[anno] += prezzo2

#Unisco i risultati tra i prestiti calcolati inizialmente con gli anni disburse e expiration uguali e quelli appena calcolati
risultato1 = risultato.reset_index()
for x in risultato1.itertuples():
    diz[x.disburse_time] += x.loan_amount

for key, value in diz.items():
    print(key , value)

2011 631678.3440311765
2012 125342923.52066368
2013 217257342.55946922
2014 281156636.59374964
2015 298671774.38838565
2016 280009692.63317835
2017 309897651.48112667
2018 105690340.99636738
